# For running inference on the TF-Hub module


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.

In [ ]:
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image

In [ ]:
!pip install Pillow

In [ ]:
import numpy as np
from PIL import Image, ImageColor, ImageDraw, ImageFont, ImageOps

# For measuring the inference time.

In [ ]:
import time

In [ ]:
print(tf.__version__)

In [ ]:
print(tf.test.gpu_device_name())

# Definition Function

In [ ]:
def display_image(image):
    fig = plt.figure(figsize=(20,15))
    plt.grid(False)
    plt.imshow(image)
    
def download_and_resize_image(url,new_width=256,new_height=256,display=False):
    _, filename = tempfile.mkstemp(suffix=".jpg")
    response = urlopen(url)
    image_data = response.read()
    image_data = BytesIO(image_data)
    pil_image = Image.open(image_data)
    pil_image =ImageOps.fit(pil_image,(new_width,new_height),image.ANTIALIAS)
    pil_image_rgb = pil_image.conver("RGB")
    pil_image_rgb.save(filename,format="JPEG",quality=90)
    print("Image downloaded to %s." %filename)
    if disply:
        display_image(pil_image)
    return filename
'''
Aniti-aliasing이란?
- 위신호 제거는 높은 해상도의 신호를 낮은해상도에서 나타낼때 생기는 계단 현상을 최소화하는 방법

pil_image.save의 quality란?
quality는 1부터 95까지의 숫자로 지정가능하며, defaultsms 75이다.
95이상은 지양하며, 100으로 지정할경우 압축이 되지 않아 알고리즘이 작동하지 않고, 파일 또한 image quality를 얻기 힘들고, 용량이 크다
'''

def draw_bounding_box_on_image(image,
                              ymin,
                              xmin,
                              ymax,
                              xmax,
                              color,
                              font,
                              thickness=4
                              display_str_list=()):
    #이미지에 bounding box를 그려넣는 과정
    draw = ImageDraw.Draw(image)
    im_width,im_height = image.size
    (left,right,top,bottom) = (xmin*im_width, xmax*im_width,
                              ymin*im_height,m xmax*im_height)
    #크기를 1이라고 가정했을 경우 x or y 값이 소수점으로 지정될 경우 이미지의 상하좌우의 위치를 설정 가능하다.
    draw.line([(left,top), (left,bottom),(right,bottom),(right,top),(left,top)],
              width=thickness,
              fill=color) #좌상 -> 좌하 -> 우하 -> 우상 -> 좌상으로 그리고 선의 길이는 두껍게, 컬러로 표시
    
    # 상단에 총 높이를 표시하고 싶을때아래의 코드 추가
    # 만약 box가 기존 높이보다 초과될경우 박스에 경고문 추가
    display_str_heigths = [font.getsize(ds)[1] for ds in display_str_list]
    # 마진을 0.05x 만큼 표시
    total_display_str_height = (1+2*0.05) * sum(display_str_heights)
    
    if top > total_display_str_height:
        text_bottom = top
    else :
        text_bottom = top + total_display_str_height
    
    #이미지를 밑에서부터 위로 출력
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin
        
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    #detection된 object 표시하는 function
    colors = list(ImageColor.colormap.values())
    
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",25)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()
    
    for i in range(min(boxes.shape[0],max_boxes)):
        if scores[i] >- min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%",format(class_name[i].decode("ascii"),int(100*scores[i]))
            color = colors[hash(class_names[i] % len(colors))]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
            image_pil,
            ymin,
            xmin,
            ymax,
            xmax,
            color,
            font,
            display_str_list=[display_str])
            np.copyto(image,np.array(image_pil))
            return image

# Using Module

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg" 
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

In [ ]:
# Using Resnet
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

detector = hub.load(module_handle).signatures['default']

In [ ]:
# Load image
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [ ]:
# Detector Running
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)

In [ ]:
run_detector(detector, downloaded_image_path)